In [30]:
import torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from model1 import model3


In [31]:
batch_size = 16
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, 
                                       transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                          num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                      transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [32]:
# net = nn.Sequential(nn.Conv2d(3, 64, 3), nn.BatchNorm2d(64), nn.ReLU())
net = model3()
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

In [33]:
for epoch in range(2):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()
   
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        if (i+1) % 500 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/500))
            running_loss = 0.

print('Finished Training')

[1,   500] loss: 2.006
[1,  1000] loss: 1.722
[1,  1500] loss: 1.566
[1,  2000] loss: 1.436
[1,  2500] loss: 1.331
[1,  3000] loss: 1.207
[2,   500] loss: 1.129
[2,  1000] loss: 1.047
[2,  1500] loss: 0.993
[2,  2000] loss: 0.956
[2,  2500] loss: 0.907
[2,  3000] loss: 0.881
Finished Training


In [34]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on 10000 test images: 70 %


In [35]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels.cuda())
    for i in range(labels.size(0)):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
        
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of plane : 78 %
Accuracy of   car : 79 %
Accuracy of  bird : 51 %
Accuracy of   cat : 52 %
Accuracy of  deer : 65 %
Accuracy of   dog : 53 %
Accuracy of  frog : 78 %
Accuracy of horse : 78 %
Accuracy of  ship : 85 %
Accuracy of truck : 86 %
